In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/NBChealth.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,548617649010532353,Fri Dec 26 23:13:35 +0000 2014,Ebola Exposure?: CDC Worker 'Remains Well' htt...
1,548569476795731968,Fri Dec 26 20:02:09 +0000 2014,Pregnant Woman Taken Off Life Support in Irela...
2,548533428023738368,Fri Dec 26 17:38:55 +0000 2014,Money from #IceBucketChallenge still flowing t...
3,548531135442337793,Fri Dec 26 17:29:48 +0000 2014,Caramel Apples Recalled After Listeria Deaths ...
4,548194819907284992,Thu Dec 25 19:13:24 +0000 2014,Ice Bucket Challenge Still Rains Bucks on ALS ...


In [3]:
# -------------------------------------------------
frame.shape

(4215, 3)

In [4]:
# -------------------------------------------------
frame.loc[:,'title'][0]

"Ebola Exposure?: CDC Worker 'Remains Well' http://nbcnews.to/1tmYMkv"

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)
frame.head()

http://nbcnews.to/1tmYMkv
http://nbcnews.to/1BcYvPG
http://ow.ly/GseY3
http://nbcnews.to/1EqpPjj
http://nbcnews.to/1thdMAd
http://ow.ly/GpzmS
http://nbcnews.to/1CLABvX
http://nbcnews.to/1vbYiYB
http://nbcnews.to/1EdtTDx
http://nbcnews.to/1zgmtY8
http://nbcnews.to/1CDgVKG
http://nbcnews.to/1zuy6Qa
http://nbcnews.to/13BkCEA
http://nbcnews.to/16NjUWM
http://nbcnews.to/1ztxbzv
http://nbcnews.to/1ADCoE6
http://nbcnews.to/1B17DXv
http://nbcnews.to/1B0cRTp
http://nbcnews.to/1t22uzR
http://nbcnews.to/1znY0Fm
http://nbcnews.to/1AMM1yo
http://nbcnews.to/1x90YgE
http://nbcnews.to/13rkdob
http://nbcnews.to/1z8qKg6
http://nbcnews.to/1CncO5i
http://nbcnews.to/1AqCGOg
http://nbcnews.to/1wOCXJS
http://nbcnews.to/1wyA1kF
http://nbcnews.to/1C9CKEm
http://nbcnews.to/1Gz4YrQ
http://nbcnews.to/1zFqgku
http://nbcnews.to/1H04M3c
http://nbcnews.to/1zCSIoa
http://nbcnews.to/1zCSJIP
http://nbcnews.to/1GvQOYp
http://nbcnews.to/13fPjPk
http://nbcnews.to/13eeS3x
http://nbcnews.to/1zxmFF4
http://nbcnews.to/1w0qaPx


http://nbcnews.to/1fkrCbf
http://nbcnews.to/1d5HfiN
http://nbcnews.to/MEmlRh
http://nbcnews.to/1h0CPOd
http://nbcnews.to/1ct3H8r
http://nbcnews.to/NduOeo
http://nbcnews.to/OdilZe
http://nbcnews.to/1eUdf9g
http://nbcnews.to/1d1P6h2
http://nbcnews.to/1eTuYh2
http://nbcnews.to/1fhGDe4
http://nbcnews.to/1jHsOWE
http://nbcnews.to/OabLTi
http://nbcnews.to/1fg37Mm
http://nbcnews.to/1m7ErHZ
http://nbcnews.to/1gCfZgd
http://nbcnews.to/1gBI2wr
http://nbcnews.to/1j8WfRT
http://nbcnews.to/1jOC0vn
http://nbcnews.to/1e4irXT
http://nbcnews.to/1jBpZX5
http://nbcnews.to/1jNtr3W
http://nbcnews.to/1kPbW3r
http://nbcnews.to/1eJhioB
http://nbcnews.to/1bFPO9d
http://nbcnews.to/1nGZv6U
http://nbcnews.to/1dD1yrU
http://nbcnews.to/1gujGoe
http://nbcnews.to/1dYRvZH
http://nbcnews.to/1dYRvZG
http://nbcnews.to/1gHGVum
http://nbcnews.to/1eCgYYN
http://nbcnews.to/1gmh2Rt
http://nbcnews.to/1f5mLL4
http://nbcnews.to/1cIBfw8
http://nbcnews.to/1jiq0z1
http://nbcnews.to/1gh32bA
http://on.today.com/1gFNvR0
http://nbcnews

http://nbcnews.to/14kiUqw
http://nbcnews.to/1dMWO4j
http://nbcnews.to/1dMKUY4
http://nbcnews.to/1ebr7PN
http://nbcnews.to/1eatfqU
http://nbcnews.to/1eaqciv
http://nbcnews.to/1dIfBxC
http://nbcnews.to/17CqAFk
http://on.today.com/17Ab64E
http://nbcnews.to/17pTKFH
http://nbcnews.to/17pSkuQ
http://nbcnews.to/17pRy10
http://on.today.com/195Fdyj
http://on.today.com/173ruEO
http://nbcnews.to/143cQ5A
http://bit.ly/1a1gEqQ
http://nbcnews.to/1dspUpg
http://nbcnews.to/1dspShb
http://nbcnews.to/17hQUo5
http://nbcnews.to/17hOzJX
http://nbcnews.to/17hOyFI
http://on.today.com/1905fD5
http://nbcnews.to/15eZNit
http://nbcnews.to/15eZN21
http://nbcnews.to/17c4Pw0
http://nbcnews.to/17c2XDo
http://nbcnews.to/14z6CsH
http://nbcnews.to/18WaMuf
http://nbcnews.to/18VQ4uI
http://on.today.com/18Wj3hQ
http://nbcnews.to/18VMssE
http://nbcnews.to/1djc08V
http://on.today.com/19TUFSu
http://on.today.com/18daQDM
http://nbcnews.to/17QZHcQ
http://nbcnews.to/17R4BXl
http://nbcnews.to/17R4zP2
http://nbcnews.to/15hu4uc
ht



http://nbcnews.to/1yRvoCt
Terminally Ill Basketball Player Signs Up for Hospice Care http://nbcnews.to/1yRvpWZ


http://nbcnews.to/1yRvpWZ
Growth in U.S. Health Spending the Lowest On Record http://nbcnews.to/1tO0CV1


http://nbcnews.to/1tO0CV1
Obamacare Bump: 10 Million Got Insurance, Survey Shows http://nbcnews.to/1yOq4yU


http://nbcnews.to/1yOq4yU
New Labels to Clarify Prescribing in Pregnancy http://nbcnews.to/1zRebaB


http://nbcnews.to/1zRebaB
IBM Helps You Donate Computer Power to Fight Ebola http://nbcnews.to/1vjzrBJ


http://nbcnews.to/1vjzrBJ
Divorce Rates Are Lower in the U.S. Why? http://nbcnews.to/1HZHrSH


http://nbcnews.to/1HZHrSH
Dr. Nancy Snyderman 'Sorry' For Violating Ebola Quarantine http://nbcnews.to/1yKH1dv


http://nbcnews.to/1yKH1dv
Radiation Danger? Kids May Get Unneeded X-Rays http://nbcnews.to/1vgVQQe


http://nbcnews.to/1vgVQQe
8 Percent of Americans Have Depression But Few Are Treated http://nbcnews.to/1vgVQQ5


http://nbcnews.to/1vgVQQ5
FDA Reconsiders 


http://nbcnews.to/1vNiuTE
Five-Time Champ Races While 8 Months Pregnant http://nbcnews.to/VuFSIK


http://nbcnews.to/VuFSIK
FDA Approves Inhalable Form of Insulin http://nbcnews.to/1yVqqEz


http://nbcnews.to/1yVqqEz
Migraines and Men: Why I Laughed When Told to 'Man Up' http://nbcnews.to/1pXDYv4


http://nbcnews.to/1pXDYv4
Did You Know? Even a Base Tan Is Bad for You http://nbcnews.to/1qjK7n6


http://nbcnews.to/1qjK7n6
Unkindest Cut: Vasectomy Has No Place in Plea Deal http://nbcnews.to/1pXfyle


http://nbcnews.to/1pXfyle
More Than 80,000 Die Yearly From Excessive Drinking http://nbcnews.to/Tpv0Kn


http://nbcnews.to/Tpv0Kn
Hospitals Using Credit Card Data to Track Patients http://nbcnews.to/1qPdP3a


http://nbcnews.to/1qPdP3a
A Leap of Faith: Patients Try Lab-Grown Organs http://nbcnews.to/1wF0qez


http://nbcnews.to/1wF0qez
Strep Amputee Will Get Rare Double Arm Transplant http://nbcnews.to/1lTG2m0


http://nbcnews.to/1lTG2m0
NY Tourist in '02 Bubonic Plague Scare Dies http://nbcn

...

A kidney for $10,000? Paying donors actually pays off, new study finds http://nbcnews.to/1aJRirW


http://nbcnews.to/1aJRirW
Opinion: Why selling kidneys still won't work http://nbcnews.to/1gMMU40


http://nbcnews.to/1gMMU40
FDA proposes strict new  safety rules for animal food http://nbcnews.to/1gNN3nV


http://nbcnews.to/1gNN3nV
http://Healthcare.gov will work smoothly by end of November, government pledges http://nbcnews.to/1aJRqYn


http://Healthcare.gov
FDA approves more powerful painkiller http://nbcnews.to/1aJYFQ4


http://nbcnews.to/1aJYFQ4
Favre's 'scary' memory lapses re-open NFL concussion questions http://nbcnews.to/1gNY60o


http://nbcnews.to/1gNY60o
North Dakota Catholics warned of possible hepatitis exposure from bishop during communion http://nbcnews.to/1aKyUPF


http://nbcnews.to/1aKyUPF
Tweeting bra exposed: Genuine support or publicity lift? http://nbcnews.to/1gNQScA


http://nbcnews.to/1gNQScA
A kidney for $10,000? Paying donors actually pays off, new study finds ht

http://nbcnews.to/1eW2N7c
VA: Aaron Alexis never sought mental health treatment http://nbcnews.to/1aNPPmP


http://nbcnews.to/1aNPPmP
Texas boy's death reignites debate over EpiPens in schools http://on.today.com/1eqiL8E


http://on.today.com/1eqiL8E
Cat poop parasite controls minds early -- and permanently, study finds http://nbcnews.to/1eqbPbs


http://nbcnews.to/1eqbPbs
E-readers may help dyslexics read more easily http://on.today.com/1dpYw86


http://on.today.com/1dpYw86
Expect bump in health spending next year, federal government forecasts http://nbcnews.to/1dpV1OK


http://nbcnews.to/1dpV1OK
It's never too early for Botox, doctor says http://on.today.com/1dpRlwz


http://on.today.com/1dpRlwz
America's sneeziest cities: 15 worst U.S. cities for fall allergies http://on.today.com/188nTK6


http://on.today.com/188nTK6
NYT: Patients fear drug cost spikes under health law http://nbcnews.to/1aRrVui


http://nbcnews.to/1aRrVui
Texas boy dies from attack by ants during game http://nbcnew


http://nbcnews.to/10UgFlp
CDC: 2012 was deadliest year for West Nile in US http://nbcnews.to/18GaQiV


http://nbcnews.to/18GaQiV
UN urges: Eat more insects! (Seriously) http://nbcnews.to/18G9OUh


http://nbcnews.to/18G9OUh
Some cancer patients may face high drug costs under health law http://nbcnews.to/13euqwO


http://nbcnews.to/13euqwO
Agent Orange liked with aggressive prostate cancer http://nbcnews.to/18FLcuQ


http://nbcnews.to/18FLcuQ
Rumors, grief and questions: A virus ravages a Saudi family http://nbcnews.to/13e3lKj


http://nbcnews.to/13e3lKj
Rare disorder leaves woman lost in familiar places http://on.today.com/YEwKiB


http://on.today.com/YEwKiB
Pediatricians take on gun lobby � carefully http://nbcnews.to/YEk7nC


http://nbcnews.to/YEk7nC
WHO: New SARS-like virus can probably pass person to person http://nbcnews.to/13qsl4o


http://nbcnews.to/13qsl4o
WHO: News SARS-like virus can probably pass person to person http://nbcnews.to/13qrdxR


http://nbcnews.to/13qrdxR
Mom's vi



http://nbcnews.to/WdoVw1
25 states to start testing new Medicaid models http://nbcnews.to/UNrj02


http://nbcnews.to/UNrj02
More recalls of US-made pet jerky treats, FDA says http://nbcnews.to/YJb5Sh


http://nbcnews.to/YJb5Sh
Flu vaccine barely worked in elderly this year http://nbcnews.to/YJaT5n


http://nbcnews.to/YJaT5n
Feds indict 4 in salmonella outbreak http://nbcnews.to/YIZgeZ


http://nbcnews.to/YIZgeZ
Women deluge hotline in probe of gynecologist http://nbcnews.to/YIMCwz


http://nbcnews.to/YIMCwz
Why young smokers should quit before turning 44 http://on.today.com/Xn1oMs


http://on.today.com/Xn1oMs
Hold the fries: We're eating less fast food http://on.today.com/XmYUO1


http://on.today.com/XmYUO1
Docs list tests, treatments you don't need http://nbcnews.to/UKXtJI


http://nbcnews.to/UKXtJI
Stop calling yourself old, says new study http://on.today.com/Vx511I


http://on.today.com/Vx511I
Florida governor expands Medicaid http://nbcnews.to/XNsfNV


http://nbcnews.to/XNsfNV
Sc

,tweetid,date,title,url
0,548617649010532353,Fri Dec 26 23:13:35 +0000 2014,Ebola Exposure?: CDC Worker 'Remains Well',http://nbcnews.to/1tmYMkv
1,548569476795731968,Fri Dec 26 20:02:09 +0000 2014,Pregnant Woman Taken Off Life Support in Ireland,http://nbcnews.to/1BcYvPG
2,548533428023738368,Fri Dec 26 17:38:55 +0000 2014,Money from #IceBucketChallenge still flowing t...,http://ow.ly/GseY3
3,548531135442337793,Fri Dec 26 17:29:48 +0000 2014,Caramel Apples Recalled After Listeria Deaths,http://nbcnews.to/1EqpPjj
4,548194819907284992,Thu Dec 25 19:13:24 +0000 2014,Ice Bucket Challenge Still Rains Bucks on ALS,http://nbcnews.to/1thdMAd


In [6]:
# -------------------------------------------------
frame.loc[:,'title'][0]

"Ebola Exposure?: CDC Worker 'Remains Well' "

In [7]:
# -------------------------------------------------
# Xpath issues
list_scrapt = ['//article[@class="articleBody"]/div/div[3]/p/text()',
               '',
               ''
               ]

In [8]:
# -------------------------------------------------
# Scrapt form web
df_temp = pd.DataFrame()
list_url = frame['url'].values

In [13]:
# -------------------------------------------------
# Run for this jupyter notebook
# -------------------------------------------------
# Run for this jupyter notebook
for key, url in enumerate(['http://nbcnews.to/1CLABvX','http://nbcnews.to/1t22uzR']):
    
    try:
        # Start the connection
        res = requests.get(url)

        # Initialize XPath
        xpath_selector = Selector(text=res.text)

        # Get return content as text
        text_html = res.text

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
    #         soup = BeautifulSoup(res.content, 'lxml')
            xpath_selector = Selector(text=text_html)

            # Scrapt content with XPath
            list_scrapt = scrapt_w_xpath(list_scrapt)
            print(list_scrapt)
            count_xp_scrapt = len(list_scrapt)

            if count_xp_scrapt:

                # Add new columns
                if is_attempt:
                    df_temp = pd.DataFrame(columns = list(list_scrapt.keys()))
                    frame = pd.concat([frame, df_temp])
                    print(frame.columns)
                    is_attempt = False

                # Add to dataFrame
                for k_, v_ in list_scrapt.items():
                    if v_:
                        frame[k_].iloc[key] = v_

            else:
                print("Return list is emptied ! ({})".format(url))
        else:
            print("Nothing is archived because of {} return {} Errors".format(url, res.status_code))
    except:
        print(url)
        pass

Header:   200
IN SCRAPT W XPath
{'contents': '', 'tags': '', 'author': ''}
http://nbcnews.to/1CLABvX
Header:   200
IN SCRAPT W XPath
http://nbcnews.to/1t22uzR


In [10]:
# -------------------------------------------------
frame.head()

,tweetid,date,title,url
0,548617649010532353,Fri Dec 26 23:13:35 +0000 2014,Ebola Exposure?: CDC Worker 'Remains Well',http://nbcnews.to/1tmYMkv
1,548569476795731968,Fri Dec 26 20:02:09 +0000 2014,Pregnant Woman Taken Off Life Support in Ireland,http://nbcnews.to/1BcYvPG
2,548533428023738368,Fri Dec 26 17:38:55 +0000 2014,Money from #IceBucketChallenge still flowing t...,http://ow.ly/GseY3
3,548531135442337793,Fri Dec 26 17:29:48 +0000 2014,Caramel Apples Recalled After Listeria Deaths,http://nbcnews.to/1EqpPjj
4,548194819907284992,Thu Dec 25 19:13:24 +0000 2014,Ice Bucket Challenge Still Rains Bucks on ALS,http://nbcnews.to/1thdMAd


In [11]:
# -------------------------------------------------
frame.shape

(4215, 4)